This notebook allows the reproduce **Table 3** of the paper "Covering the Combinatorial Design Space of Multiplex CRISPR/Cas Experiments in Plants" by Kirsten Van Huffel, Michiel Stock, Thomas Jacobs, Tom Ruttink and Bernard De Baets.

#### Load packages

In [1]:
using Random 
using Plots  
using Distributions 
using LinearAlgebra
using Combinatorics
using BioCCP


(process:21104): GLib-GIO-WARNING **: 16:41:24.866: Unexpectedly, UWP app `HaukeGtze.7-ZipFileManagerUnofficial_1.2107.2.0_x64__6bk20wvc8rfx2' (AUMId `HaukeGtze.7-ZipFileManagerUnofficial_6bk20wvc8rfx2!zFM.exe') supports 36 extensions but has no verbs


#### Load BioCCP- and simulation-based framework

In [2]:
include("MultiplexCrisprDOE.jl");

#### Set seed for reproducibility

In [3]:
Random.seed!(1)

TaskLocalRNG()

#### Calculate expected plant library size for full coverage and measure execution time for both approaches

1) CRISPR/Cas experiment targeting single gene knockouts

In [4]:
include("DefaultParameters_k=1.jl");

In [5]:
@time simulate_Nₓ₁(x, 
                                g, 
                                r, 
                                n_gRNA_total, 
                                p_gRNA_freq, 
                                p_gRNA_edit, ϵ_KO; iter=500)

  0.950617 seconds (3.42 M allocations: 320.631 MiB, 6.37% gc time, 68.76% compilation time)


(107.79, 35.02220738360221)

In [6]:
@time BioCCP_Nₓ₁(x, 
                                g, 
                                r, 
                                n_gRNA_total, 
                                p_gRNA_freq, 
                                p_gRNA_edit, ϵ_KO)

  0.547845 seconds (2.54 M allocations: 128.299 MiB, 8.11% gc time, 99.69% compilation time)


(110, 38)

2) Multiplex CRISPR/Cas experiment targeting pairwise combinations of gene knockouts

In [7]:
include("DefaultParameters_k=2.jl");

In [8]:
@time simulate_Nₓ₂(x, 
                                g, 
                                r, 
                                n_gRNA_total, 
                                p_gRNA_freq, 
                                p_gRNA_edit, ϵ_KO; iter=500)

  7.235114 seconds (22.79 M allocations: 6.973 GiB, 13.57% gc time, 3.99% compilation time)


(2785.944, 653.5442303210796)

In [9]:
@time BioCCP_Nₓ₂(x, 
                                g, 
                                r, 
                                n_gRNA_total, 
                                p_gRNA_freq, 
                                p_gRNA_edit, ϵ_KO)

  0.774351 seconds (1.72 M allocations: 113.445 MiB, 2.84% gc time, 97.17% compilation time)


(2731, 641)

3) Multiplex CRISPR/Cas experiment targeting triple combinations of gene knockouts

In [10]:
include("DefaultParameters_k=3.jl");

In [ ]:
@time simulate_Nₓ₃(x, 
                                g, 
                                r, 
                                n_gRNA_total, 
                                p_gRNA_freq, 
                                p_gRNA_edit, ϵ_KO; iter=500)

In [ ]:
@time BioCCP_Nₓ₃(x, 
                                g, 
                                r, 
                                n_gRNA_total, 
                                p_gRNA_freq, 
                                p_gRNA_edit, ϵ_KO)